In [1]:
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import Input, Conv1D, GlobalMaxPool1D 
from keras.layers import BatchNormalization, concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.losses import CategoricalCrossentropy, MeanSquaredError
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras import optimizers

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from pathlib import Path
from tqdm import tqdm
import re

In [2]:
ROOT = Path.cwd().parent
INPUT = ROOT/'input'
DATA = INPUT/'feedback-prize-english-language-learning'
WORK = ROOT/'working'

In [3]:
target_col = ['cohesion', 'syntax', 'vocabulary',
              'phraseology', 'grammar', 'conventions']
max_len = 1440 #200
batch_size = 32
epochs = 10
max_words =30000
num_classes = 9

In [4]:
def decontractions(phrase):
    phrase = re.sub(r"wan\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", "not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def preprocess(text):
    preprocessed = []
    for sentence in tqdm(text.values):
        sentence = str(sentence)
        sent = sentence.replace('\n\n', ' ')
        sent = decontractions(sent)
        preprocessed.append(sent.lower().strip())
    return preprocessed

In [5]:
def model_init(max_len, max_words):
    input_1 = Input(shape=(max_len,))
    embed = Embedding(input_dim=max_words,
                      output_dim=128,
                      input_length=max_len)(input_1)
                 # weights=[embedding_matrix],
                #  trainable=False)(input_1)

    branches = []
    x = Dropout(0.2)(embed)
    for i in range(2, 6):
        branch = Conv1D(128, i,
                        padding = 'valid',
                        activation = 'relu')(x)
        branch = GlobalMaxPool1D()(branch)
        branches.append(branch)

    x = concatenate(branches, axis=1)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(9)(x)
    output = Activation('softmax')(x)

    model = Model(inputs = [input_1],
                  outputs = [output])
    
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = 'adam',
                  metrics = ['accuracy'])
    return model

In [6]:
df_train = pd.read_csv(DATA/'train.csv')
df_train.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [7]:
df_train.full_text = preprocess(df_train.full_text)

token = Tokenizer()
token.fit_on_texts(df_train.full_text)
df_train['token'] = token.texts_to_sequences(df_train.full_text)
x_train = pad_sequences(df_train.token, maxlen=max_len, padding='post')

100%|██████████| 3911/3911 [00:00<00:00, 39135.60it/s]


In [8]:
for col in tqdm(target_col):
    y_train = to_categorical(df_train[col]*2-2, num_classes)
    model_save = WORK/f'model_{col}.h5'
    checkpoint = ModelCheckpoint(model_save,
                                 monitor = 'val_accuracy',
                                 save_best_only = True,
                                 verbose = 1)
    model = model_init(max_len, max_words)
    history = model.fit(x_train, y_train,
                        batch_size = batch_size,
                        epochs = epochs,
                        validation_split = 0.1,
                        callbacks = [checkpoint],
                        verbose = 1)
    model.load_weights(model_save)
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = 'adam',
                  metrics = ['accuracy'])
    history_2 = model.fit(x_train, y_train,
                          batch_size = batch_size,
                          epochs = epochs,
                          validation_split = 0.2,
                          callbacks = [checkpoint],
                          verbose = 1)

  0%|          | 0/6 [00:00<?, ?it/s]2022-10-02 21:42:35.274713: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-10-02 21:42:35.490028: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
110/110 [==============================] - 84s 758ms/step - loss: 1.7324 - accuracy: 0.2654 - val_loss: 1.6733 - val_accuracy: 0.2755

Epoch 00001: val_accuracy improved from -inf to 0.27551, saving model to /kaggle/working/model_cohesion.h5
Epoch 2/10
110/110 [==============================] - 83s 756ms/step - loss: 1.6649 - accuracy: 0.3041 - val_loss: 1.6376 - val_accuracy: 0.3240

Epoch 00002: val_accuracy improved from 0.27551 to 0.32398, saving model to /kaggle/working/model_cohesion.h5
Epoch 3/10
110/110 [==============================] - 82s 747ms/step - loss: 1.5916 - accuracy: 0.3572 - val_loss: 1.6387 - val_accuracy: 0.2985

Epoch 00003: val_accuracy did not improve from 0.32398
Epoch 4/10
110/110 [==============================] - 83s 755ms/step - loss: 1.4945 - accuracy: 0.4135 - val_loss: 1.5698 - val_accuracy: 0.3393

Epoch 00004: val_accuracy improved from 0.32398 to 0.33929, saving model to /kaggle/working/model_cohesion.h5
Epoch 5/10
110/110 [==============

 17%|█▋        | 1/6 [26:54<2:14:32, 1614.53s/it]


Epoch 00010: val_accuracy did not improve from 0.44061
Epoch 1/10
110/110 [==============================] - 88s 792ms/step - loss: 1.7061 - accuracy: 0.3055 - val_loss: 1.6462 - val_accuracy: 0.3214

Epoch 00001: val_accuracy improved from -inf to 0.32143, saving model to /kaggle/working/model_syntax.h5
Epoch 2/10
110/110 [==============================] - 88s 799ms/step - loss: 1.6366 - accuracy: 0.3169 - val_loss: 1.5906 - val_accuracy: 0.3444

Epoch 00002: val_accuracy improved from 0.32143 to 0.34439, saving model to /kaggle/working/model_syntax.h5
Epoch 3/10
110/110 [==============================] - 86s 782ms/step - loss: 1.5631 - accuracy: 0.3643 - val_loss: 1.5445 - val_accuracy: 0.3495

Epoch 00003: val_accuracy improved from 0.34439 to 0.34949, saving model to /kaggle/working/model_syntax.h5
Epoch 4/10
110/110 [==============================] - 88s 804ms/step - loss: 1.4593 - accuracy: 0.4260 - val_loss: 1.5003 - val_accuracy: 0.3673

Epoch 00004: val_accuracy improved from

 33%|███▎      | 2/6 [54:45<1:49:51, 1647.84s/it]

...


Epoch 00010: val_accuracy did not improve from 0.61303
Epoch 1/10
110/110 [==============================] - 89s 805ms/step - loss: 1.8105 - accuracy: 0.2393 - val_loss: 1.6900 - val_accuracy: 0.2500

Epoch 00001: val_accuracy improved from -inf to 0.25000, saving model to /kaggle/working/model_grammar.h5
Epoch 2/10
110/110 [==============================] - 89s 813ms/step - loss: 1.7089 - accuracy: 0.2913 - val_loss: 1.6525 - val_accuracy: 0.2857

Epoch 00002: val_accuracy improved from 0.25000 to 0.28571, saving model to /kaggle/working/model_grammar.h5
Epoch 3/10
110/110 [==============================] - 91s 824ms/step - loss: 1.6357 - accuracy: 0.3424 - val_loss: 1.6244 - val_accuracy: 0.2806

Epoch 00003: val_accuracy did not improve from 0.28571
Epoch 4/10
110/110 [==============================] - 90s 818ms/step - loss: 1.5270 - accuracy: 0.4186 - val_loss: 1.5791 - val_accuracy: 0.3010

Epoch 00004: val_accuracy improved from 0.28571 to 0.30102, saving model to /kaggle/workin

 83%|████████▎ | 5/6 [2:18:54<28:01, 1681.17s/it]


Epoch 00010: val_accuracy did not improve from 0.58748
Epoch 1/10
110/110 [==============================] - 93s 835ms/step - loss: 1.7562 - accuracy: 0.2810 - val_loss: 1.6853 - val_accuracy: 0.2551

Epoch 00001: val_accuracy improved from -inf to 0.25510, saving model to /kaggle/working/model_conventions.h5
Epoch 2/10
110/110 [==============================] - 90s 823ms/step - loss: 1.6926 - accuracy: 0.2967 - val_loss: 1.6684 - val_accuracy: 0.2959

Epoch 00002: val_accuracy improved from 0.25510 to 0.29592, saving model to /kaggle/working/model_conventions.h5
Epoch 3/10
110/110 [==============================] - 90s 816ms/step - loss: 1.6537 - accuracy: 0.3160 - val_loss: 1.6552 - val_accuracy: 0.2832

Epoch 00003: val_accuracy did not improve from 0.29592
Epoch 4/10
110/110 [==============================] - 90s 819ms/step - loss: 1.5669 - accuracy: 0.3660 - val_loss: 1.6413 - val_accuracy: 0.3138

Epoch 00004: val_accuracy improved from 0.29592 to 0.31378, saving model to /kaggl

100%|██████████| 6/6 [2:47:18<00:00, 1673.02s/it]


Epoch 00010: val_accuracy did not improve from 0.57344


In [9]:
test = pd.read_csv(DATA/'test.csv')
sample = pd.read_csv(DATA/'sample_submission.csv')
sample.text_id = test.text_id
test.head()

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [10]:
test.full_text = preprocess(test.full_text)
x_test = token.texts_to_sequences(test.full_text)
x_test = pad_sequences(x_test, padding='post', maxlen=max_len)

100%|██████████| 3/3 [00:00<00:00, 7589.21it/s]


In [11]:
def label_transform(pred):
    labels = []
    for i in range(len(pred)):
        max_p = max(pred[i])
        for j in range(num_classes):
            if max_p == pred[i][j]:
                ind = (j + 2) / 2
                break
        labels.append(ind)
    return labels

for col in tqdm(target_col):
    model.load_weights(model_save)
    test_pred = model.predict(x_test,
                              batch_size = batch_size,
                              verbose = 1)
    sample[col] = label_transform(test_pred)

  0%|          | 0/6 [00:00<?, ?it/s]

1/1 [==============================] - 0s 160ms/step


 17%|█▋        | 1/6 [00:00<00:01,  4.86it/s]

1/1 [==============================] - 0s 44ms/step


 50%|█████     | 3/6 [00:00<00:00,  8.21it/s]

1/1 [==============================] - 0s 44ms/step


 83%|████████▎ | 5/6 [00:00<00:00,  8.84it/s]

1/1 [==============================] - 0s 45ms/step


100%|██████████| 6/6 [00:00<00:00,  8.62it/s]


In [12]:
sample.to_csv('submission.csv', index=False)